In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import urllib3
urllib3.disable_warnings()
import numpy as np

# 60 city agencies

In [2]:
# url="https://a860-openrecords.nyc.gov/request/view_all"
# pagination=requests.get(url)
# page_no=BeautifulSoup(pagination.text)

In [3]:
final_dates=pd.read_csv("data/dates.csv")

base="https://a860-openrecords.nyc.gov/search/requests?query=&title=on&agency_request_summary=on&open=on&closed=on&"
sum_counter=0
table_list=[]

for link in final_dates.dates: 
    URLs=f"{base}{link}&agency_ein=&size=50&tz_name=America%2FNew_York&start="

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko)\
        Chrome/91.0.4472.77 Safari/537.36',
    }
    response = requests.get(URLs,headers=headers,verify=False)
    

    total_entries=int(response.json()['total'])
    sum_counter=sum_counter+total_entries
    
    print(link, sum_counter)

    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko)\
        Chrome/91.0.4472.77 Safari/537.36',
    }


    for i in np.arange(0, total_entries, 50):
        response = requests.get(f"{base}{link}&agency_ein=&size=50&tz_name=America%2FNew_York&start={i}",
                                headers=headers,verify=False)
        doc=BeautifulSoup(response.json()['results'])
        for tr in doc.find_all('tr'):
            table_dict={}
            table_dict['status']=tr.find_all('td')[0].text.strip()
            table_dict['foil_id']=tr.find_all('td')[1].text.strip()
            table_dict['date_submitted']=tr.find_all('td')[2].text.strip()
            table_dict['title']=tr.find_all('td')[3].text.strip()
            table_dict['ass_agency']=tr.find_all('td')[4].text.strip()
            table_dict['due_date']=tr.find_all('td')[5].text.strip()
            table_dict['href_id']=tr.find_all('td')[3].find('a')['href'].strip()
            table_list.append(table_dict)
    


df=pd.DataFrame(table_list)

# url pattern: https://a860-openrecords.nyc.gov/request/view/FOIL-2022-057-03974

df['foil_link']='https://a860-openrecords.nyc.gov'+df.href_id
df['date_submitted']=pd.to_datetime(df.date_submitted)
df['due_date']=pd.to_datetime(df.due_date)
df=df.sort_values("date_submitted").reset_index(drop=True)


date_rec_from=04%2F01%2F2021&date_rec_to=04%2F30%2F2021 5813
date_rec_from=06%2F01%2F2021&date_rec_to=06%2F30%2F2021 11789
date_rec_from=09%2F01%2F2021&date_rec_to=09%2F30%2F2021 16817
date_rec_from=11%2F01%2F2021&date_rec_to=11%2F30%2F2021 22182
date_rec_from=04%2F01%2F2020&date_rec_to=04%2F30%2F2020 25553
date_rec_from=06%2F01%2F2020&date_rec_to=06%2F30%2F2020 30345
date_rec_from=09%2F01%2F2020&date_rec_to=09%2F30%2F2020 36292
date_rec_from=11%2F01%2F2020&date_rec_to=11%2F30%2F2020 41444
date_rec_from=04%2F01%2F2019&date_rec_to=04%2F30%2F2019 46072
date_rec_from=06%2F01%2F2019&date_rec_to=06%2F30%2F2019 50560
date_rec_from=09%2F01%2F2019&date_rec_to=09%2F30%2F2019 55014
date_rec_from=11%2F01%2F2019&date_rec_to=11%2F30%2F2019 60080
date_rec_from=04%2F01%2F2018&date_rec_to=04%2F30%2F2018 62184
date_rec_from=06%2F01%2F2018&date_rec_to=06%2F30%2F2018 66830
date_rec_from=09%2F01%2F2018&date_rec_to=09%2F30%2F2018 70355
date_rec_from=11%2F01%2F2018&date_rec_to=11%2F30%2F2018 74357
date_rec_

In [4]:
df.to_csv("data/foil_records_20180101_20220321.csv", index=False)